In [2]:
%load_ext autoreload
%autoreload 2
import os
from os.path import join, expanduser
from tqdm import tqdm
import pandas as pd
import joblib
import sys
import numpy as np
from copy import deepcopy
from neuro.features.feat_select import get_alphas
from neuro.features.qa_questions import get_questions

In [3]:
sparse_feats_dir = expanduser('~/mntv1/deep-fMRI/qa/sparse_feats_shared/')
os.listdir(sparse_feats_dir)
qa_sparse_feats_dir = join(
    sparse_feats_dir,
    'qa_embedder___qa_questions_version=v3_boostexamples___ensemble1')
# 'eng1000___qa_questions_version=v1___mistralai-Mistral-7B-Instruct-v0.2')
# alphas = np.logspace(0, -3, 20)
alphas = get_alphas('qa_embedder')
# alphas = get_alphas('eng1000')

In [4]:
seeds = range(5)
nonzeros = pd.DataFrame(index=seeds, columns=alphas).astype(float)
enets = pd.DataFrame(index=seeds, columns=alphas)
fnames = os.listdir(qa_sparse_feats_dir)
for seed in tqdm(seeds):
    for alpha in tqdm(alphas):
        template = f'seed={seed}___feature_selection_frac=0.50___feature_selection_alpha={alpha:.2e}.joblib'
        if template in fnames:
            coef_enet = joblib.load(join(qa_sparse_feats_dir, template))
            coef_enet_selected = deepcopy(
                np.any(np.abs(coef_enet) > 0, axis=0).squeeze())
            enets.loc[seed, alpha] = coef_enet_selected
            nonzeros.loc[seed, alpha] = coef_enet_selected.sum()
# template = f'seed={seed}___feature_selection_frac=0.50___feature_selection_alpha={feature_selection_alpha:.2e}.joblib'
# os.listdir(qa_sparse_feats_dir)

100%|██████████| 5/5 [00:05<00:00,  1.08s/it]


In [5]:
# v3_boost_examples has 674, eng1000 has 985
nonzeros.columns.name = 'alpha'
nonzeros.index.name = 'seed'
# nonzeros.columns = np.arange(len(nonzeros.columns))
# enets.columns = nonzeros.columns
# nonzeros.columns = nonzeros.columns.round(4)

coefs_stable_dict = {}
coefs_all_dict = {}
# add row for fracs
for col in nonzeros.columns:
    coefs_list = enets[col]
    coefs_list = coefs_list[coefs_list.notna()]

    if len(coefs_list) > 0:
        # get fraction of times each element is True
        coefs_arr = np.vstack(coefs_list.values)
        coefs_all = coefs_arr.max(axis=0)
        coefs_arr = coefs_arr.min(axis=0)
        nonzeros_stable = np.sum(coefs_arr)
    else:
        nonzeros_stable = np.nan
    nonzeros.loc['stable', col] = nonzeros_stable
    coefs_all_dict[col] = deepcopy(coefs_all)
    coefs_stable_dict[col] = deepcopy(coefs_arr)

display(
    nonzeros
    .style
    .background_gradient(cmap='viridis', axis=None)
    .format('{:.0f}')
)

alpha,0.483293,0.400000,0.335982,0.280000,0.233572,0.162378,0.112884,0.078476,0.054556
seed,,,,,,,,,
0,15,25,38,67,105,218,380,533,626
1,15,26,36,65,101,214,370,525,626
2,15,24,35,63,101,218,392,524,630
3,12,25,37,63,103,214,383,518,623
4,13,27,35,68,97,217,378,520,615
stable,10,21,28,44,71,147,299,468,595


### Top questions

In [6]:
questions = np.array(get_questions('v3_boostexamples', full=True))
qs_prev = []
for i in range(4):
    print(i, 'num questions:', len(
        questions[list(coefs_stable_dict.values())[i]]))

    print('----STABLE---')
    qs = sorted(questions[list(coefs_stable_dict.values())[i]].tolist())
    qs_unstable = sorted(questions[list(coefs_all_dict.values())[i]].tolist())
    display([q for q in qs if q not in qs_prev])
    for q in qs_prev:
        if not q in qs:
            print('[DROPPED]', q)

    print('----UNSTABLE---')
    display([q for q in qs_unstable if not q in qs])

    qs_prev = qs

0 num questions: 10
----STABLE---


['Does the input include a philosophical or reflective thought?',
 'Does the sentence contain a proper noun?',
 'Does the sentence describe a physical action?',
 'Does the sentence describe a relationship between people?',
 'Does the sentence include dialogue or thoughts directed towards another character?',
 'Does the sentence involve a description of an interpersonal misunderstanding or dispute?',
 'Does the sentence involve a description of physical environment or setting?',
 'Does the sentence involve the mention of a specific object or item?',
 'Does the sentence reference a specific location or place?',
 'Is time mentioned in the input?']

----UNSTABLE---


['Does the input involve planning or organizing?',
 'Does the sentence describe a personal or social interaction that leads to a change or revelation?',
 'Does the sentence describe a visual experience or scene?',
 'Does the sentence include technical or specialized terminology?',
 'Does the sentence involve a discussion about personal or social values?',
 'Does the sentence involve an expression of personal values or beliefs?',
 'Does the sentence mention a specific location or place?',
 'Is the input about a relationship with another person?']

1 num questions: 21
----STABLE---


...

['Does the input describe a sensory experience?',
 'Does the input include a comparison or metaphor?',
 'Does the input involve a debate or controversial topic?',
 'Does the sentence describe a physical sensation?',
 'Does the sentence describe a sensory experience?',
 'Does the sentence include a direct speech quotation?',
 'Does the sentence include a number or statistic?',
 'Does the sentence involve a description of an emotional reaction to a surprise or unexpected event?',
 'Does the sentence involve a description of an interpersonal misunderstanding or dispute?',
 'Does the sentence involve moral reasoning?',
 'Does the sentence involve spatial reasoning?',
 'Does the sentence use irony or sarcasm?',
 'Does the text describe a journey?',
 'Does the text include a planning or decision-making process?',
 'Is the input about a relationship with another person?',
 'Is the sentence conveying a strategic or tactical thought by the narrator?',
 'Is the sentence expressing skepticism or 

3 num questions: 44
----STABLE---


['Does the input discuss a societal issue or social justice topic?',
 'Does the input include a comparison or metaphor?',
 'Does the sentence describe a physical sensation?',
 'Does the sentence describe a routine activity?',
 'Does the sentence describe a sensory experience?',
 'Does the sentence express a philosophical or existential query or observation?',
 'Does the sentence include a direct speech quotation?',
 'Does the sentence include a number or statistic?',
 'Does the sentence include a personal anecdote or story?',
 'Does the sentence involve moral reasoning?',
 'Does the sentence involve spatial reasoning?',
 'Does the sentence use a unique or unusual word?',
 'Does the sentence use irony or sarcasm?',
 'Does the text describe a journey?',
 'Is the input about a practical joke or prank?',
 'Is the sentence a command?',
 'Is the sentence designed to persuade or convince?',
 'Is there mention of a city, country, or geographic feature?']

[DROPPED] Does the sentence express the narrator's opinion or judgment about an event or character?
[DROPPED] Does the sentence include dialogue or thoughts directed towards another character?
----UNSTABLE---


['Does the input contain a measurement?',
 'Does the input describe a sensory experience?',
 'Does the input describe a specific emotion in detail?',
 'Does the input involve a debate or controversial topic?',
 'Does the input involve a philosophical question or theory?',
 'Does the input mention a routine or daily activity?',
 'Does the sentence convey a decision or choice made by the narrator?',
 'Does the sentence convey a sense of urgency or haste?',
 'Does the sentence describe a moment of relief or resolution of tension?',
 'Does the sentence describe a personal reflection or thought?',
 'Does the sentence describe a physical sensation (e.g., touch, taste)?',
 'Does the sentence describe a significant personal interaction?',
 'Does the sentence describe an activity related to daily life or routine?',
 'Does the sentence describe an experience of learning or gaining new knowledge?',
 "Does the sentence express the narrator's opinion or judgment about an event or character?",
 'Doe

# Manually clean up questions

In [7]:
questions = np.array(get_questions('v3_boostexamples', full=True))

In [8]:
df = pd.DataFrame(questions)
df.index.name = 'question_num'
df['stable_idx'] = 10
coefs_stable_vals = list(coefs_stable_dict.values())
for i in range(len(coefs_stable_vals) - 1, -1, -1):
    df.loc[coefs_stable_vals[i], 'stable_idx'] = i
df = df.sort_values(by='stable_idx')

# display full questions no truncation
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
    display(df)

,0,stable_idx
question_num,,
404,Does the sentence describe a relationship between people?,0
379,Does the sentence contain a proper noun?,0
447,Does the sentence reference a specific location or place?,0
625,Does the sentence involve a description of an interpersonal misunderstanding or dispute?,0
667,Does the sentence involve the mention of a specific object or item?,0
375,Is time mentioned in the input?,0
628,Does the sentence involve a description of physical environment or setting?,0
436,Does the sentence include dialogue or thoughts directed towards another character?,0
57,Does the input include a philosophical or reflective thought?,0


In [9]:
df = df.sort_index()

In [10]:
df

,0,stable_idx
question_num,,
0,Does the input contain a measurement?,5
1,Does the input contain a number?,7
2,Does the input contain a question?,8
3,Does the input contain advice or recommendations?,7
4,Does the input contain an expression of anger?,6
...,...,...
669,Does the sentence mention a family member?,5
670,Does the sentence mention a past or historical...,5
671,Does the sentence mention a specific location ...,1


In [12]:
s = ''
for i in range(len(df)):
    s += f'{i}. {questions[i]}\n'

In [15]:
s

"0. Does the input contain a measurement?\n1. Does the input contain a number?\n2. Does the input contain a question?\n3. Does the input contain advice or recommendations?\n4. Does the input contain an expression of anger?\n5. Does the input contain emotional words?\n6. Does the input contain humor or sarcasm?\n7. Does the input contain negativity?\n8. Does the input contain repetition?\n9. Does the input describe a body of water?\n10. Does the input describe a form of local or community service?\n11. Does the input describe a moment of cultural shock or adaptation?\n12. Does the input describe a moment of realization or epiphany?\n13. Does the input describe a moment of teamwork or collaboration?\n14. Does the input describe a nighttime setting?\n15. Does the input describe a problem or challenge?\n16. Does the input describe a scenic view or panorama?\n17. Does the input describe a scientific experiment or discovery?\n18. Does the input describe a sensory experience?\n19. Does the in

In [ ]:
'''Below is a numbered list of questions. Group and list any questions (along with their numbers) that are very similar to each other'''